In [30]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('text_collector.csv')
df.head()
def extract_filename(path):
    """
    Extracts the filename without extension from a given file path.

    Parameters:
    path (str): The file path from which to extract the filename.

    Returns:
    str: The filename without its extension.
    """
    import os
    # Extract the base name of the file (e.g., 'AZ_Glendale.txt')
    base_name = os.path.basename(path)
    # Split the base name by '.' and return the first part (e.g., 'AZ_Glendale')
    return os.path.splitext(base_name)[0]
df['Location'] = df['Location'].apply(extract_filename)
df.head()

import ast

columns_to_apply = ['Weights_Matches', 'Weight Values', 'Weight Result','+-250_Text']
for column in columns_to_apply:
    df[column] = df[column].apply(ast.literal_eval)


# Define the unpack_nested_list function
def unpack_nested_list(nested_list):
    while isinstance(nested_list, list) and len(nested_list) == 1:
        nested_list = nested_list[0]
    return nested_list


#df.to_csv('beforeoutput.csv', index=False)
df['Weight Values'] = df['Weight Values'].apply(unpack_nested_list)

for index, row in df.iterrows():
    if len(row['Weight Values']) != len(row['Weights_Matches']):
        if len(row['Weight Values']) > 1:
            df.at[index, 'Weight Values'] = row['Weight Values'][-1]

#df.to_csv('afteroutput.csv', index=False)

df.head()
new_rows = []


    
# Iterate through the DataFrame
for index, row in df.iterrows():
    # If 'Num_Matches_Found' is greater than 1, append the duplicated rows
    if row['Num_Matches_Found'] > 1:
        for i in range(row['Num_Matches_Found']):
            copied_row = row.copy()
            copied_row['Num_Matches_Found'] = 1
            copied_row['+-250_Text'] = copied_row['+-250_Text'][i]
            new_rows.append(copied_row)
    else:
        new_rows.append(row)      
        
# Reset the original row's 'Num_Matches_Found' to 1
new_rows[-(row['Num_Matches_Found'])]['Num_Matches_Found'] = 1

# Create a new DataFrame from the new rows list
df = pd.DataFrame(new_rows)

# Reset index to maintain consistency
df.reset_index(drop=True, inplace=True)
#df.to_csv('output1.csv', index=False)


# Assuming df is your DataFrame
new_rows = []

# Iterate through the DataFrame
for index, row in df.iterrows():
    num_matches=0
    num_matches = len(row['Weights_Matches'])
    len_weight_val_num_matches = len(row['Weight Values'])
    len_weight_match_num_matches = len(row['Weights_Matches'])
    if num_matches > 1:
        for i in range(num_matches):
            copied_row = row.copy()
            copied_row['Weights_Matches'] = copied_row['Weights_Matches'][i]
            copied_row['Weight Values'] = copied_row['Weight Values'][i]
            copied_row['Weight Result'] = copied_row['Weight Result'][i]
            new_rows.append(copied_row)
    else:
        copied_row = row.copy()
        copied_row['Weight Values'] = []
        new_rows.append(copied_row)


# Create a new DataFrame from the new rows list
df = pd.DataFrame(new_rows)
# Reset index to maintain consistency
df.reset_index(drop=True, inplace=True)
df
df.to_csv('out_text_collector_analyzer.csv', index=False)